In [ ]:
!pip install tensorflow-gpu
!pip install transformers
!pip install tokenizers
!pip install torch

     |████████████████████████████████| 394.3MB 38kB/s 
     |████████████████████████████████| 1.8MB 6.9MB/s 
     |████████████████████████████████| 2.9MB 34.9MB/s 
     |████████████████████████████████| 890kB 28.3MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=9df008d5c3c11ce36a7e43f4c3eb8c30eb9d5f388c8c4e0552aee44c3556df4c
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import torch
from transformers import LineByLineTextDataset, DataCollatorForLanguageModeling
from transformers import BertTokenizer, BertForMaskedLM
from transformers import Trainer, TrainingArguments

In [ ]:
def load_dataset(train_path, tokenizer):
    train_dataset = LineByLineTextDataset(
        tokenizer=tokenizer,
        file_path=train_path,
        block_size=128)

    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer, mlm=True,  mlm_probability=0.15
    )
    return train_dataset, data_collator

In [ ]:
tokenizer = BertTokenizer.from_pretrained("readerbench/RoBERT-small")
model = BertForMaskedLM.from_pretrained("readerbench/RoBERT-small")

training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/Grade/model-new/",  # The output directory
    overwrite_output_dir=True,  # overwrite the content of the output directory
    num_train_epochs=1,
    per_device_train_batch_size=64,
    save_steps=10_000,
    save_total_limit=2,
    warmup_steps=500,  # number of warmup steps for learning rate scheduler
    prediction_loss_only=True,
)

train_dataset, data_collator = load_dataset("/content/drive/MyDrive/Grade/dataset/text-bert.txt", tokenizer)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset
)

trainer.train()

trainer.save_model("/content/drive/MyDrive/Grade/model-best/")

Some weights of the model checkpoint at readerbench/RoBERT-base were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
/usr/local/lib/python3.6/dist-packages/transformers/data/datasets/language_modeling.py:128: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/language-modeling/r